In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import zipfile
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from sklearn.metrics import confusion_matrix, precision_score, recall_score


# Define the path to the zip files
train_zip_path = r'C:\Users\Ryan\OneDrive\Documents\Computer Vision\CV Discussion3\train.zip'
test_zip_path = r'C:\Users\Ryan\OneDrive\Documents\Computer Vision\CV Discussion3\test.zip'

# Define the extraction directories
train_dir = r'C:\Users\Ryan\OneDrive\Documents\Computer Vision\CV Discussion3\train_data'

# Extract the training zip file
with zipfile.ZipFile(train_zip_path, 'r') as zip_ref:
    zip_ref.extractall(train_dir)


print("Extraction complete!")

Extraction complete!


In [4]:

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())




[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 3271711838149554432
xla_global_id: -1
]


In [ ]:
# Define the correct extraction directory and subdirectory
train_dir = r'C:\Users\Ryan\OneDrive\Documents\Computer Vision\CV Discussion3\train_data\train'

# Initialize lists for file paths and labels
file_paths = []
labels = []

# Iterate through the files in the subdirectory and label them
for filename in os.listdir(train_dir):
    if filename.lower().startswith('cat'):
        labels.append(0)
        file_paths.append(os.path.join(train_dir, filename))
    elif filename.lower().startswith('dog'):
        labels.append(1)
        file_paths.append(os.path.join(train_dir, filename))

# Check if file_paths and labels have been populated
print(f"Number of file paths: {len(file_paths)}")
print(f"Number of labels: {len(labels)}")

# Create DataFrame if there are data
if file_paths and labels:
    df = pd.DataFrame({
        'filename': file_paths,
        'class': labels
    })

    print(f"DataFrame shape: {df.shape}")
    print(df.head())
else:
    print("No images found or labels not assigned correctly.")

In [ ]:
# Convert numeric labels to strings
df['class'] = df['class'].map({0: 'cat', 1: 'dog'})

# Verify the changes
print(df.head())
print(df['class'].value_counts())

In [ ]:
# Define image size and batch size
IMG_HEIGHT = 224
IMG_WIDTH = 224
BATCH_SIZE = 32

# Create ImageDataGenerator instances
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range = 50,
    zoom_range = .2,
    horizontal_flip = True,
    brightness_range = [.5, 1.7],
    validation_split=0.2
)

# Training generator
train_generator = datagen.flow_from_dataframe(
    dataframe=df,
    directory=None,  # Use None since file paths are absolute
    x_col='filename',
    y_col='class',
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training',
    shuffle = True
)

# Validation generator
validation_generator = datagen.flow_from_dataframe(
    dataframe=df,
    directory=None,  # Use None since file paths are absolute
    x_col='filename',
    y_col='class',
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation',
    shuffle = True
)

print("Custom data generators created!")

In [ ]:
# Load the pre-trained MobileNetV2 model
base_model = tf.keras.applications.MobileNetV2(
    weights='imagenet',  # Load weights pre-trained on ImageNet
    include_top=False,  # Exclude the final dense layers
    input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)  # Input shape
)

# Freeze the base model
base_model.trainable = False

# Build the new model on top of the pre-trained model
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),  # Reduce dimensions from (H, W, C) to (C)
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.3),  # Dropout with 30% probability
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.3),  # Dropout with 30% probability
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.3),  # Dropout with 30% probability
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.3),  # Dropout with 30% probability
    layers.Dense(1, activation='sigmoid')  # Output layer with sigmoid activation for binary classification
])


#This week we are going to change solvers in the middle of training
#Compile and train the model with the Adam optimizer
model.compile(
    optimizer='adam',  # Initial optimizer
    loss='binary_crossentropy',
    metrics=['accuracy']
)

#Train for the first half of the epochs
history_adam = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=5,  # Train for 5 epochs with Adam
    initial_epoch=0  # Start from the beginning
)

#Switch the optimizer to SGD
model.compile(
    optimizer=tf.keras.optimizers.SGD(learning_rate=0.001, momentum=0.9),  # New optimizer
    loss='binary_crossentropy',
    metrics=['accuracy']
)

#Continue training with SGD optimizer
history_sgd = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=10,  # Continue training for 5 more epochs (total 10)
    initial_epoch=5  # Start from the 5th epoch
)

In [ ]:
test_dir = r'C:\Users\Ryan\OneDrive\Documents\Computer Vision\CV Discussion3\test_data\test'

# List all test images with valid extensions
test_image_files = [f for f in os.listdir(test_dir) if f.endswith(('.jpg', '.jpeg', '.png'))]

# Create DataFrame with a 'filename' column
test_df = pd.DataFrame({
    'filename': [os.path.join(test_dir, f) for f in test_image_files]  # Full paths to test images
})

# Debug: Print the DataFrame to ensure it's correct
print("DataFrame Columns:", test_df.columns)
print(test_df.head())  # Check the first few rows to verify the filenames

In [ ]:

# Define the ImageDataGenerator
test_datagen = ImageDataGenerator(rescale=1./255)

# Test data generator
test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    directory=None,  # Use None if file paths are absolute
    x_col='filename',
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=1,  # Typically batch size of 1 for predictions
    class_mode=None,  # No labels for test data
    shuffle=False  # Ensure the order is maintained for results
)

#reset the test generator
test_generator.reset()

# Make predictions
predictions = model.predict(test_generator)

# Convert predictions to probabilities
predicted_probabilities = predictions.flatten()

# Convert probabilities to binary class predictions
predicted_classes = (predicted_probabilities > 0.5).astype(int)
